# Web Scraping METROCUADRADO

## Librerías

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

## Requerimiento a la pagína

## Selección de grilla

In [4]:
driver = webdriver.Chrome()
driver.get('https://www.metrocuadrado.com/venta/?search=form')
# Get the page source after interactions
page_source = driver.page_source
# Parse the page source with Beautiful Soup
soup = BeautifulSoup(page_source, 'lxml')

driver.quit()

In [5]:
house_grill = soup.find('ul', class_ = 'browse-results-list')

In [7]:
house_grill

<ul class="Ul-sctud2-0 jyGHXP realestate-results-list browse-results-list" role="navigation" tag="nav" toggleable="false"><li class="sc-gPEVay dibcyk"><div class="sc-jqCOkK bpoZJE RecordCard-sc-1tx60lw-0 kerSYW CardResultDesktop-sc-9359pw-0 iUYuci d-none d-sm-block mb-3 CardResultDesktop-sc-9359pw-0 iUYuci d-none d-sm-block mb-3 card"><ul class="sc-iRbamj kYQRXi inline-list-grid" role="navigation" tag="nav" toggleable="false"><li class="sc-gPEVay dibcyk card-result-img"><a class="sc-bdVaJa ebNrSm" data-flk-success="atNodeInserted424" data-gtm-businesstype="venta" data-gtm-city="Chía" data-gtm-neighborhood="NA" data-gtm-position="1" data-gtm-productcategory="venta - Casa" data-gtm-productname="Casa en Venta, Vereda Tiquiza, Chía" data-gtm-productprice="680000000" data-gtm-productsku="11369-M4772309" data-gtm-projectname="Ocasional" data-gtm-propertytype="Casa" data-gtm-zone="" href="/inmueble/venta-casa-chia-na-3-habitaciones-3-banos-2-garajes/11369-M4772309" target="_blank"><span class

In [10]:
house_grill.find('div', class_ = 'card-header')

<div class="card-header"><ul class="sc-iRbamj kYQRXi inline-list-items" role="navigation" tag="nav" toggleable="false"><li class="sc-gPEVay dibcyk"><div><p class="sc-fMiknA ZUMHA text-project text-capitalize"></p></div></li></ul><a class="sc-bdVaJa ebNrSm" data-gtm-businesstype="venta" data-gtm-city="Chía" data-gtm-neighborhood="NA" data-gtm-position="1" data-gtm-productcategory="venta - Casa" data-gtm-productname="Casa en Venta, Vereda Tiquiza, Chía" data-gtm-productprice="680000000" data-gtm-productsku="11369-M4772309" data-gtm-projectname="Ocasional" data-gtm-propertytype="Casa" data-gtm-zone="" href="/inmueble/venta-casa-chia-na-3-habitaciones-3-banos-2-garajes/11369-M4772309" target="_blank"><h2 class="sc-dxgOiQ BSoGx card-title"><div>Casa en Venta, Vereda Tiquiza, Chía</div></h2></a></div>

In [82]:
df = pd.DataFrame()

## Nombre Publicación

In [83]:
name_houses_list = []
price_houses = []
area_houses = []
rooms_houses = []
baths_houses = []

for name_houses in house_grill:

    name = name_houses.find('h2', class_ = 'card-title').text
    name_houses_list.append(name)

    info = name_houses.find_all('p', class_ = 'card-subitem')
    
    price_houses.append(info[0].text)
    area_houses.append(info[1].text)
    rooms_houses.append(info[2].text)
    baths_houses.append(info[3].text)


df = pd.DataFrame(name_houses_list, columns=['Nombre_publicacion'])
df['Precio_inmueble'] = price_houses
df['Area_construida'] = area_houses
df['Habitaciones'] = rooms_houses
df['Banios'] = baths_houses
print(df.head(5))

                       Nombre_publicacion Precio_inmueble Area_construida  \
0  Casa en Venta, colinas de suba, Bogot…  $2.600.000.000          290 m²   
1          Casa en Venta, Canelón, Cajicá  $1.250.000.000          274 m²   
2         Casa en Venta, CHUNUGUA, Cajicá  $2.200.000.000          312 m²   
3    Apartamento en Venta, La Colina Cam…  $1.050.000.000       158.32 m²   
4   Casa en Venta, Vereda Cerca de Piedr…  $1.850.000.000          455 m²   

  Habitaciones Banios  
0            3      4  
1            5      5  
2            3      5  
3            3      5  
4            4      5  


## Multiples páginas

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [84]:
driver = webdriver.Chrome()
driver.get('https://www.metrocuadrado.com/venta/?search=form')
# Get the page source after interactions
page_source = driver.page_source
# Parse the page source with Beautiful Soup
soup = BeautifulSoup(page_source, 'lxml')


df_final = pd.DataFrame()


try:
    # Espera hasta que la página cargue completamente
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "browse-results-list")))

    # Ciclo para navegar a través de las páginas
    current_page = 1
    while True:
        
        # Extrae datos aquí, por ejemplo, nombres de productos
        
        df = pd.DataFrame()
        
        #elements = driver.find_elements(By.CLASS_NAME, "browse-results-list")
        house_grill = soup.find('ul', class_ = 'browse-results-list')
        name_houses_list = []
        price_houses = []
        area_houses = []
        rooms_houses = []
        baths_houses = []

        for name_houses in house_grill:

            name = name_houses.find('h2', class_ = 'card-title').text
            name_houses_list.append(name)

            info = name_houses.find_all('p', class_ = 'card-subitem')
            
            price_houses.append(info[0].text)
            area_houses.append(info[1].text)
            rooms_houses.append(info[2].text)
            baths_houses.append(info[3].text)


        df = pd.DataFrame(name_houses_list, columns=['Nombre_publicacion'])
        df['Precio_inmueble'] = price_houses
        df['Area_construida'] = area_houses
        df['Habitaciones'] = rooms_houses
        df['Banios'] = baths_houses
        df['Pagina'] = current_page
        #print(df.head(5))
        print("Página:",current_page)

        df_final = pd.concat([df_final, df], ignore_index=True)

        # Intenta encontrar el número de la siguiente página y hacer clic en él
        next_page = current_page + 1
        page_elements = driver.find_elements(By.CLASS_NAME, "pagination")

        find_element = driver.find_element(By.CLASS_NAME, "pagination")

        # Ejecutar JavaScript para hacer scroll hasta el elemento
        driver.execute_script("arguments[0].scrollIntoView(true);", find_element)        

        found_next_page = False 

        time.sleep(10)       

        # # Iterar sobre cada elemento para imprimir su texto
        for page_element in page_elements:
            for page in page_element.find_elements(By.CLASS_NAME, 'page-link'):               
                if page.tag_name == 'a':
                    if page.text == str(next_page):                   
                        print(page)
                        page.click()
                        found_next_page = True
                        current_page = next_page
                        time.sleep(5)  # Espera para asegurar que la página se haya cargado completamente
                        break
        
        if not found_next_page:
            print("Reached last page.")
            break
# except Exception as e:
#     print(f"An error occurred: {e}")
finally:
    # Cierra el navegador
    driver.quit()
    


Página: 1
<selenium.webdriver.remote.webelement.WebElement (session="c5a18207eee97689aa0712b0f6fe7842", element="f.2710670978B5ADF462C1662A76BE0EB1.d.EC389DC4EABD220472734E8BDC6DD18A.e.144")>
Página: 2
<selenium.webdriver.remote.webelement.WebElement (session="c5a18207eee97689aa0712b0f6fe7842", element="f.2710670978B5ADF462C1662A76BE0EB1.d.EC389DC4EABD220472734E8BDC6DD18A.e.145")>
Página: 3
<selenium.webdriver.remote.webelement.WebElement (session="c5a18207eee97689aa0712b0f6fe7842", element="f.2710670978B5ADF462C1662A76BE0EB1.d.EC389DC4EABD220472734E8BDC6DD18A.e.146")>
Página: 4
<selenium.webdriver.remote.webelement.WebElement (session="c5a18207eee97689aa0712b0f6fe7842", element="f.2710670978B5ADF462C1662A76BE0EB1.d.EC389DC4EABD220472734E8BDC6DD18A.e.147")>
Página: 5


KeyboardInterrupt: 

In [91]:
df_final.duplicated().value_counts()

False    245
True       5
Name: count, dtype: int64